In [4]:
import requests
import time
import calendar
import re
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [5]:
def nielsenKoreaCrawlling(startYear, startMonth, startDay, url, topN, dates, channels, programs, ratings):
    # 넬슨코리아 시청률 페이지 접속
    driver = webdriver.Chrome('./drivers/chromedriver.exe')
    driver.implicitly_wait(3)
    driver.get(url)
    driver.maximize_window()

    # 검색 가능한 년도, 월, 일 알아내기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 년도
    sYear = soup.find('select', {'name' : 'sYear'})
    sYearTotal = [int(re.sub('[^0-9]', '', i.get_text())) for i in sYear.find_all('option')]
    # startYear 선택
    if startYear in sYearTotal:
        sYearIndex = sYearTotal.index(startYear) + 1
        driver.find_element_by_xpath('//*[@id="select"]/option[' + str(sYearIndex) + ']').click()
    else:
        return "No startYear!"
    # startMonth 선택
    if (startMonth>0)&(startMonth<=12):
        driver.find_element_by_xpath('//*[@id="select2"]/option[' + str(startMonth) + ']').click()
    else:
        return "No startMonth!"
    
    # 검색버튼 클릭
    driver.find_element_by_xpath('//*[@id="sub_body"]/table[1]/tbody/tr/td/form/table/tbody/tr/td[2]/img').click()
    # 전국버튼 클릭
    driver.find_element_by_xpath('//*[@id="sub_body"]/table[2]/tbody/tr/td/table/tbody/tr/td[1]').click()

    # dayCalc 구하기
    dayCalc = (datetime.now() - datetime(startYear, startMonth, startDay)).days
    print(str(startYear) + '-' + str(startMonth) + '-' + str(startDay) + ' ~ ' + datetime.now().strftime('%Y-%m-%d') + ':' +  str(dayCalc))
    cnt = 0
    isbreak = True
    
    # crawlling
    while(1):
        try:
            driver.find_element_by_xpath('//*[@id="select"]').click()
            driver.find_element_by_xpath('//*[@id="select"]/option[' + str(sYearIndex) + ']').click()
            while(1):
                driver.find_element_by_xpath('//*[@id="select2"]').click()
                driver.find_element_by_xpath('//*[@id="select2"]/option[' + str(startMonth) + ']').click()
                while (1):
                    driver.find_element_by_xpath('//*[@id="select1"]').click()
                    driver.find_element_by_xpath('//*[@id="select1"]/option[' + str(startDay) + ']').click()
                    driver.find_element_by_xpath('//*[@id="sub_body"]/table[1]/tbody/tr/td/form/table/tbody/tr/td[2]/img').click()  # 검색 버튼
                    driver.find_element_by_xpath('//*[@id="sub_body"]/table[2]/tbody/tr/td/table/tbody/tr/td[1]').click()   # 전국 버튼

                    # 현재 날짜 페이지 html parsing
                    html2 = driver.page_source
                    soup2 = BeautifulSoup(html2, 'html.parser')

                    # dates append
                    date = soup2.find('td', {'class' : 'ranking_date'})
                    dateText = [re.sub("[^0-9.]", '', date.get_text()) for i in range(topN)]
                    dates.append(dateText)

                    # channels append
                    channel = soup2.find_all('td', {'class' : 'tb_txt_center'})
                    channelText = [channel[2*i+1].get_text().replace('\t', '') for i in range(topN)]
                    channels.append(channelText)
                        
                    # programs append
                    program = soup2.find_all('td', {'class' : 'tb_txt'})
                    programText = [program[i].get_text().replace('\t', '') for i in range(topN)]
                    programs.append(programText)

                    # ratings append
                    rating = soup2.find_all('td', {'align' : 'center'})
                    ratingText = [re.sub('[^0-9.]', '', rating[i].get_text()) for i in range(1, topN+1)]
                    ratings.append(ratingText)

                    print(cnt)
                    cnt += 1
                    if (cnt >= dayCalc):
                        isbreak = False
                        break

                    startDay += 1
                    if startDay > calendar.monthrange(sYearTotal[sYearIndex-1], startMonth)[1]:
                        startDay = 1
                        break
                    
                if (isbreak == False):
                    break
                startMonth += 1
                if (startMonth > 12):
                    startMonth = 1
                    break
                
            if (isbreak == False):
                break     
            sYearIndex += 1        
            
        except Exception as e:
            errorDate = datetime(sYearTotal[sYearIndex-1], startMonth, startDay)
            nextDate = errorDate + timedelta(days=1)
            print("=================================Error!================================")
            print(e)
            print("Error cnt : ", cnt)
            print("Error date : ", errorDate)
            print("Next date : ", nextDate)
            print("=======================================================================")
            nielsenKoreaCrawlling(nextDate.year, nextDate.month, nextDate.day, driver.current_url, len(dateText), dates, channels, programs, ratings)
            break
        
    return dates, channels, programs, ratings

In [41]:
url = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=3_1&area=01&begin_date=202006'
dates = []
channels = []
programs = []
ratings = []
dates, channels, programs, ratings = nielsenKoreaCrawlling(2018, 1, 1, url, 10, dates, channels, programs, ratings)

2018-1-1-2020-06-24:905
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

In [45]:
print(len(dates))
print(len(channels))
print(len(programs))
print(len(ratings))

904
903
903
903


In [48]:
del(dates[418])

In [52]:
datesFinal = sum(dates, [])
channelsFinal = sum(channels, [])
programsFinal = sum(programs, [])
ratingsFinal = sum(ratings, [])

print(len(datesFinal))
print(len(channelsFinal))
print(len(programsFinal))
print(len(ratingsFinal))

9030
9030
9030
9030


In [53]:
df = pd.DataFrame({"날짜" : datesFinal, "채널" : channelsFinal, "프로그램" : programsFinal, "시청률" : ratingsFinal})
df.to_csv("20200623 닐슨코리아시청률(케이블).csv", encoding='utf-8', index=False)
df.to_excel("20200623 닐슨코리아시청률(케이블).xlsx")

In [5]:
df = pd.read_csv('20200623 닐슨코리아시청률(케이블).csv')
df.head()

,날짜,채널,프로그램,시청률
0,2018.01.01,tvN,사장이더많이먹는강식당,3.450
1,2018.01.01,tvN,월화드라마(막돼먹은영애씨16<본>),2.694
2,2018.01.01,tvN,사장이더많이먹는강식당,2.679
3,2018.01.01,tvN,사장이더많이먹는강식당,2.296
4,2018.01.01,tvN,사장이더많이먹는강식당,2.215


In [6]:
day = ['월', '화', '수', '목', '금', '토', '일']
dayList = []
for index, date in enumerate(df.날짜):
    splitDate = date.split('.')
    dayIndex = datetime(int(splitDate[0]), int(splitDate[1]), int(splitDate[2])).weekday()
    dayList.append(day[dayIndex])

df['요일'] = dayList
df.head()

,날짜,채널,프로그램,시청률,요일
0,2018.01.01,tvN,사장이더많이먹는강식당,3.450,월
1,2018.01.01,tvN,월화드라마(막돼먹은영애씨16<본>),2.694,월
2,2018.01.01,tvN,사장이더많이먹는강식당,2.679,월
3,2018.01.01,tvN,사장이더많이먹는강식당,2.296,월
4,2018.01.01,tvN,사장이더많이먹는강식당,2.215,월


In [8]:
df.to_csv("20200623 닐슨코리아시청률(케이블).csv", encoding='utf-8', index=False)
df.to_excel("20200623 닐슨코리아시청률(케이블).xlsx")